In [22]:
#Import from the Keras library
from keras import models
from keras.layers import Dense, Dropout, Flatten, Input 
from keras.layers import Conv2D,  MaxPooling2D, TimeDistributed, LSTM, Conv3D, MaxPooling3D
from keras.models import Model
from keras import optimizers 
from keras import utils
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras_tqdm import TQDMNotebookCallback
#from secret import credentials

import sys
import gc

#This allows for Keras models to be saved. 
import h5py
#Other import statements 
import h5py
import random
import numpy as np
import pandas as pd
import pathlib
import cv2
import pymysql
import os

#pick which recordings you are working with
where_clause = "WHERE (r.recording_id > 1)"
#limit the data to a subset for testing
#Make empty string to have no limit
#limit = "ORDER BY RAND(1) LIMIT 3000"
limit_clause = "LIMIT 1500"

#What size should we reduce images to before learning in percent
#image_scale_percent = 1

epochs = 1000
batch_size = 32
max_accuracy = 0.90 #Quit if accuracy reaches this at any epoch

#directory where data from database is stored
cache_path = 'cache'

#File name for the statistics to be save in. Must include .txt at the end
statistics_output_file = 'statistics.sequential.output.txt'

#Must have the h5py package installed or the model will not save. This should be the path of the location you would like
#To save the model
model_file_name = 'model.output'

#The amount of frames to include in every sequence
sequence_length = 10

In [23]:
#Secrets shouldn't be in the repository
from secrets import credentials

def connect(): 
    def connect(): 
    db_host = credentials['db_host'];
    db_port = credentials['db_port'];
    db_name = credentials['db_name'];
    db_username = credentials['db_username']
    db_password = credentials['db_password']
    
    conn = pymysql.connect(db_host, user=db_username, port=db_port, passwd=db_password, db=db_name)
    return conn

In [24]:
def import_class_data(conn): 
    
    cursor = conn.cursor()
    try:
        #a dictionary that maps classes to a dictionary that maps recording_ids to lists of raw_ids
        ''' 2 classes , 2 recording ids, 15 frames
            {
                0:{
                    2:{[1,2,3,4],[2,3,4,5]},
                    3:{[6,7,8,9],[7,8,9,10]}
                },
                1:{
                    2:{[3,4,5,6],[4,5,6,7,8]},
                    3:{[8,9,10,11,12],[7,8,9,10]}
                }
            }
        '''
                    
        class_data={}
        rec_data={}
        
        recording_query = "SELECT r.id, r.recording_id, r.isCSGM FROM nicu.Video_Raw AS r JOIN nicu.Video_Generated AS g ON r.id=g.raw_id "+where_clause+" "+limit_clause
        try:
            cursor.execute(recording_query)
            for row in cursor.fetchall():
                raw_id = row[0]
                rec_id = row[1]
                csgm = row[2]
                
                if rec_id not in rec_data:
                    rec_data[rec_id] = list()
                    
                rec_data[rec_id].append([raw_id, csgm])               
                    
                        
            for rec_id in rec_data:
                raw_ids = rec_data[rec_id]
                for i in range(len(raw_ids)):
                    #This takes sequence at position n and will add the other raw_ids before n and creates a list the length
                    #of sequence_length so that if sequence_length = 3 the sequence will be [n-2,n-1,n]
                    if i >= sequence_length:
                        frame = raw_ids[i]
                        raw_id = frame[0]
                        csgm = frame[1]

                        j=0
                        sequence = list()
                        sequence.append(raw_id)
                        while j < (sequence_length-1):
                            if frame[0] - raw_ids[i-j][0] == j:
                                sequence.insert(0,raw_ids[i-j][0])
                            else: 
                                break
                            j+=1
                        
                        if csgm not in class_data:
                            class_data[csgm] = dict()
                        
                        
                        single_class_data = class_data[csgm]

                        if rec_id not in single_class_data:
                            single_class_data[rec_id] = list()    
                        
                        single_class_data[rec_id].append(sequence)
                        
                    
                        
                                                      
        except Exception as e:
            print("Error retrieving ID's", e)
            conn.rollback()
            raise e
            
        return class_data
    finally:
        cursor.close()

In [25]:
def create_array(conn, class_data, batch_sequences):     
    
    #Find the true class of the elements in the batch
    csgms = list() #The raw_id mapped to the class
    ordered_sequences = list()
    all_classes = class_data.keys()
    for a_class in all_classes:
        for recording_id, sequences in class_data[a_class].items():
            for sequence in sequences:
                if sequence in batch_sequences:
                    csgms.append(a_class)
                    ordered_sequences.append(sequence)

    
    #Retrieve the images
    
    if not os.path.exists(cache_path):
        os.mkdir(cache_path)
    
    sequence_list=list()
    csgm_list=list()
    rgb_sequence_list=list()
    d_sequence_list=list()
    
    for i in range(len(csgms)):
        sequence = ordered_sequences[i]
        the_class = csgms[i]
        image_list=[]
        rgb_image_list=list()
        d_image_list=list()
        for frame in sequence: 
            #Check the cache for the images in question
            current_rgb_input = cache_path+'/'+str(frame)+".oflow.png"
            current_d_input = cache_path+'/'+str(frame)+".dflow.png"
            if not os.path.exists(current_d_input) or not os.path.exists(current_rgb_input):
                cursor = conn.cursor()
                try:
                    image_query = "SELECT RGB_Optical_Flow, D_Depth_Flow from Video_Generated WHERE (raw_id=%s)"
                    cursor.execute(image_query, (str(frame)))
                    for row in cursor.fetchall():
                        db_rgb_img = row[0]
                        db_d_img = row[1]
                        if (db_rgb_img is not None):
                            rgb_img=cv2.imdecode(np.asarray(bytearray(db_rgb_img),dtype=np.uint8),cv2.IMREAD_UNCHANGED)
                            cv2.imwrite(current_rgb_input,rgb_img)
                            print("⇣",end="",flush=True)
                        else:
                            print("x",end="",flush=True)
                        if (db_d_img is not None):    
                            d_img=cv2.imdecode(np.asarray(bytearray(db_d_img),dtype=np.uint8),cv2.IMREAD_UNCHANGED)
                            cv2.imwrite(current_d_input,d_img)
                            print("⇣",end="",flush=True)
                        else:
                            print("x",end="",flush=True)
                except Exception as e:
                    print("Error retrieving frame",e)
                    raise e
                finally:
                    cursor.close()     
            else:
                print("o",end="",flush=True)

            rgb_img = cv2.imread(current_rgb_input)
            d_img  = cv2.imread(current_d_input)
            if rgb_img is not None and d_img is not None: 
                #Resizing the image
                #width = int(img.shape[1] * image_scale_percent / 100)
                #height = int(img.shape[0] * image_scale_percent / 100)
                #cv2.resize(img,(width,height), interpolation=cv2.INTER_CUBIC)
                rgb_image_list.append(rgb_img)
                d_image_list.append(d_img)
        
    
        if len(rgb_image_list) == sequence_length and len(d_image_list) == sequence_length:
            rgb_sequence_list.append(rgb_image_list) 
            d_sequence_list.append(d_image_list) 
            csgm_list.append(the_class)
        
    x_rgb = np.array(rgb_sequence_list)
    print(x_rgb.shape)
    x_d = np.array(d_sequence_list)
    y = np.array(csgm_list)
    return x_rgb, x_d, y

In [26]:
def create_cnn(input_shape, keys=[0,1,2], filter_info={0:[32,3]}, dropout={0:0.25}, pooling={0:2}, activation='relu', loss='mean_squared_error', final_activation='sigmoid'):    
    
    model = models.Sequential() 
    for k in keys:
        num_filters = filter_info[k][0]
        filter_size = filter_info[k][1]
        if k == keys[0]: 
            model.add(Conv2D(num_filters, (filter_size,filter_size), activation = activation, input_shape=input_shape))
        else: 
            model.add(Conv2D(num_filters, (filter_size,filter_size), activation= activation))
                
        if k in pooling.keys():           
            pool_filter_size = pooling[k]
            model.add(MaxPooling2D(pool_size=(pool_filter_size, pool_filter_size)))            
                 
        if k in dropout.keys(): 
            drop_rate = dropout[k]
            model.add(Dropout(drop_rate))

    return model

In [27]:
keys=[0,1,2]
filter_info={0:[32,3],1:[64,3],2:[128,3]}
dropout={0:0.25,1:0.25,2:0.25}
pooling={0:2,1:2,2:2}

rgb_model = create_cnn((480,640,3),
                    keys,
                    filter_info=filter_info,
                    dropout=dropout,
                    pooling=pooling,
                    loss='mean_squared_error', 
                    final_activation='sigmoid',
                    activation='relu')

depth_model = create_cnn((480,640,3),
                    keys,
                    filter_info=filter_info,
                    dropout=dropout,
                    pooling=pooling,
                    activation='relu')


rgb_input = Input(shape=(sequence_length,480,640,3))
encoded_rgb = TimeDistributed(rgb_model)(rgb_input)
        
depth_input = Input(shape=(sequence_length,480,640,3))
encoded_depth = TimeDistributed(depth_model)(depth_input)
        
merged = keras.layers.concatenate([encoded_rgb, encoded_depth],axis=4)
conv3d_1 = Conv3D(32,(3,3,3))(merged)
max3d_1 = MaxPooling3D((2,2,2))(conv3d_1)
flatten = Flatten()(max3d_1)
dense = Dense(128, activation='softmax')(flatten)
output = Dense(1, activation='sigmoid')(dense)
        

model = Model(inputs=[rgb_input, depth_input], outputs=output)

model.compile(loss='mean_squared_error', 
              optimizer=optimizers.SGD(lr=1e-4),
              metrics=['acc']) 

#Start the new output file
with open(statistics_output_file, 'w') as f:
    if 'CUDA_VISIBLE_DEVICES' not in os.environ or os.environ['CUDA_VISIBLE_DEVICES'] == None or os.environ['CUDA_VISIBLE_DEVICES'] == '-1':
        f.write('Used GPU: False\n')
    else:
        f.write('Used GPU: True\n')
    f.write("where_clause:\"{}\"\n".format(where_clause))
    f.write("limit_clause:\"{}\"\n".format(limit_clause))
    f.write("epochs:\"{}\"\n".format(epochs))
    f.write("batch_size:\"{}\"\n".format(batch_size))
    f.write("max_accuracy:\"{}\"\n\n".format(max_accuracy))
    model.summary(print_fn=lambda x: f.write("Model:\n{}\n".format(x)))

model.summary() #output for the notebook

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 10, 480, 640, 0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 10, 480, 640, 0                                            
__________________________________________________________________________________________________
time_distributed_5 (TimeDistrib (None, 10, 58, 78, 1 93248       input_5[0][0]                    
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 10, 58, 78, 1 93248       input_6[0][0]                    
__________________________________________________________________________________________________
concatenat

In [28]:
def confusion_matrix(exp_values, predicted_values):
    """
    This creates a confusion matrix with the predicted accuracy of the model.
    
    exp_values must be in the format of a list and predicted values is expected to come in the format of the ouput 
    of Keras's model.predict()
    
    The ouput is a pandas dataframe that displays a confusion matrix indicitive of the accuracy of the model along 
        with a number score which is the accuracy of the model.
    """
    predicted_values = convert_predictions(predicted_values)
    
    
    
    #Creates a DataFrame of zeros
    matrix = pd.DataFrame(np.zeros((2,2)) , ['P0','P1'], ['E0','E1'])
   
    #Caculates whether the score was right or wrong and updates the confusion matrix 
    for i in range(len(exp_values)):
        if exp_values[i] == predicted_values[i]:
            matrix.iloc[[predicted_values[i]],[predicted_values[i]]] += 1
        else:
            matrix.iloc[[predicted_values[i]],[exp_values[i]]] += 1
   
    #Calculate diagonal sum and the accuracy of the model
    #Precision (TP/TP+FPos)      Recall TP(TP+FNegative)
    diagonal_sum = 0
    for i in range(2):
        diagonal_sum += matrix.iloc[i][i]
    
    score = diagonal_sum/len(exp_values)
    
  
    return  matrix, score

In [31]:
#updated
def runTest(model,pooling, dropout, filter_info, file_name='model.txt', model_name='model', save_model=False, epochs=5, batch_size=32,max_accuracy=0.90):
    
    conn = connect()
    try:
        #recording_ids_dict = import_recording_ids(cache_path,conn)
        class_data = import_class_data(conn)
        
        all_classes = class_data.keys()

        matrices = {}
        scores = {}
        model_scores = {}
    
        #Set up to do cross-validation
        all_recording_ids = set()
        for a_class in all_classes:
            for recording_id in class_data[a_class].keys():
                all_recording_ids.add(recording_id)

        #Run one fold of cross validation
        for test_recording_id in all_recording_ids:
            #Clean up some memory by garbage collecting variables that hold data
            train_sequences = train_sequences_list = test_sequences = x_train = y_train = x_test = y_test = None
            gc.collect()
        
            print('Testing on {0:3} '.format(test_recording_id),end="")
            train_sequences = list()
            test_sequences = list()

        
            #Given the recording we are using for testing, split up the raw_ids into training and testing groups
            for recording_id in all_recording_ids:
                sequences = list()
                for a_class in all_classes:
                    for key, values in class_data[a_class].items():
                        if key == recording_id:
                            for value in values:
                                sequences.append(value)
                
                if test_recording_id == recording_id:
                    test_sequences.extend(sequences[:2])
                else: 
                    train_sequences.extend(sequences[:2])
                    
            #Create the testing set
            x_rgb_test, x_depth_test, y_test = create_array(conn,class_data,test_sequences)
            print()
            
        
            #Scaling the values to a value between 0 and 1
            x_rgb_test = x_rgb_test.astype('float32')
            x_rgb_test /= 255
            
            x_depth_test = x_depth_test.astype('float32')
            x_depth_test /= 255
        
            epoch = 0
            accuracy = 0
            no_improvement = 0
            while epoch < epochs and accuracy < max_accuracy and no_improvement < 2:
                #Split the train_ids up into batches that are randomly shuffled and iteratively train themodel
                train_sequences_list = list(train_sequences)
                random.shuffle(train_sequences_list)
                
                print("\tEpoch {0:5} {1:6} samples ".format(epoch,len(train_ids_list)),end="")
                
                #Go through a batch at a time
                for i in range(0,len(train_sequences_list), batch_size):
                    #Get the current batch
                    batch = list()
                    for j in range(i,i+batch_size):
                        if j < len(train_sequences_list):
                            batch.append(train_sequences_list[j])
                    
                    #Fetch the images for the batch
                    print(" {} ".format(i),end="",flush=True)
                    x_rgb, x_depth, y_batch = create_array(conn,class_data,batch)
                
                    #Scaling the rgb values to a value between 0 and 1
                    x_rgb = x_rgb.astype('float32')
                    x_rgb /= 255
                    x_depth = x_depth.astype('float32')
                    x_depth /= 255
                
                    print('Shape of' + str(x_rgb.shape))
                    print('Shape of' + str(x_depth.shape))
                    #Fit the model
                    model.train_on_batch([x_rgb ,x_depth], y_batch)
                print("",flush=True)
              
                #Create predictions and evaluate to find loss and accuaracy
                predict = model.predict([x_rgb_test,x_depth_test])
                model_score = model.evaluate([x_rgb_test,x_depth_test], y_test)
                print('\t\tModel was ' + str(model_score[1]) + '% accurate and exhibited an average loss of ' + str(model_score[0]) + '.')

                matrix,score = confusion_matrix(y_test, predict)
                print(str(matrix) + '\n')
                print(str(score) + '\n')

                results.append({"matrix":matrix,
                             "score":score,
                             "model_score":model_score
                            })

                #update epoch
                epoch = epoch + 1

                #update loss
                loss = model_score[0]

                #if accuracy didn't change...
                if accuracy == model_score[1]:
                    delta_loss_percentage = abs(loss - model_score[0])/loss
                    average_delta_loss_percentage = 0.9*average_delta_loss_percentage + 0.1*delta_loss_percentage
                else:
                    average_delta_loss_percentage = 1.0

                #update accuracy
                accuracy = model_score[1]

            if epoch == epochs:
                stop_reason = "Reached end of training run"
                print("Reached end of training run")
            if accuracy >= max_accuracy:
                stop_reason = "Stopped training early because max_accuracy was reached"
            if average_delta_loss_percentage < loss_threshold:
                stop_reason = "Training stopped progressing"

            print(stop_reason)
            output_file.write("{},".format(accuracy))
            output_file.write("{},".format(loss))
            output_file.write("{},".format(matrix.iloc[0][0]))
            output_file.write("{},".format(matrix.iloc[0][1]))
            output_file.write("{},".format(matrix.iloc[1][0]))
            output_file.write("{},".format(matrix.iloc[1][1]))  
            output_file.write("{}\n".format(stop_reason))
        aggregate_accuracy = []
        aggregate_loss = []
        aggregate_TN = []
        aggregate_FN = []
        aggregate_FP = []
        aggregate_TP = []
        for r in results:
            aggregate_accuracy.append(r["model_score"][1])
            aggregate_loss.append(r["model_score"][0])
            aggregate_TN.append(r["matrix"].iloc[0][0])
            aggregate_FN.append(r["matrix"].iloc[0][1])
            aggregate_FP.append(r["matrix"].iloc[1][0])
            aggregate_TP.append(r["matrix"].iloc[1][1])
        output_file.write("Across all folds:\n")
        output_file.write("Average Accuracy: {}, Standard Deviation: {}\n".format(statistics.mean(aggregate_accuracy),statistics.stdev(aggregate_accuracy)))
        output_file.write("Average Loss: {}, Standard Deviation: {}\n".format(statistics.mean(aggregate_loss),statistics.stdev(aggregate_loss)))
        output_file.write("\t\t\tEO\tE1\n")
        output_file.write("\tP0\t{}\t{}\n".format(statistics.sum(aggregate_TN),statistics.sum(aggregate_FN)))
        output_file.write("\tP0\t{}\t{}\n".format(statistics.sum(aggregate_FP),statistics.sum(aggregate_TP)))

        if save_model: 
            model.save(model)
    finally:
        conn.close()

In [32]:
runTest(model=model, file_name=statistics_output_file, 
        filter_info=filter_info,  
        dropout=dropout, 
        pooling=pooling, 
        epochs=epochs, 
        batch_size=batch_size,
        max_accuracy=max_accuracy)

Testing on   2 oooooooooooooooooooo(2, 10, 480, 640, 3)



TypeError: object of type 'NoneType' has no len()